In [5]:
import numpy as np
# Import the Ising Model Functions Class
from functions import Ising_Functions

In [6]:
# All the parameters are set up for N = 100
kT = 0.2
N = 100
# Set up a zero array to contain random spin grid
spin = [[0 for col in range(N)] for row in range(N)]
HalfPos = int(N / 2) - 1
spin[HalfPos][HalfPos] = 1
spin[HalfPos + 1][HalfPos] = 1
spin[HalfPos][HalfPos + 1] = 1
spin[HalfPos + 1][HalfPos + 1] = 1
spin[0][0] = 1
spin[0][1] = 1
spin[1][0] = 1
spin[1][1] = 1

# Reformat the array so that it is numpy friendly
spin = np.array(spin)

In [12]:
spin_array_size = np.size(spin)
# Pick a random spin in the array "spin"
linearIndex = np.random.randint(1, spin_array_size)

In [14]:
get_values = Ising_Functions(spin, linearIndex, N).Fitness_Eqn_Val()
neighbours = get_values[0]
neighbours_sqrd = get_values[1]
one_minus_neighbour = get_values[2]
neighbour_spin_product = get_values[4]
print(neighbours, neighbours_sqrd, one_minus_neighbour, neighbour_spin_product)

0 0 4 0
